# Developing an AI Application Locally

In this section, we will develop an AI-powered application that reads docx files from a designated folder, converts them into embeddings, and stores them in a vector store.

Afterward, we will use a similarity search to retrieve relevant meanings and provide contextual answers to your questions.

This application will enable you to quickly understand the essence of books and delve deeper into character development.

## Setting up

First, we will install all the necessary Python packages for loading the documents, vector store, and LLM frameworks.


pip install unstructured[docx] langcha
pip install unstructured[docx] langchain langchainhub langchain_community langchain-chroma
pip install networkx

ollama serve

## Loading the documents

It is a best practice to develop and test your code in Jupyter Notebook before creating the app.

We will load all the docx files from the folder using the DirectoryLoader.


C:/Users/rod90731/OneDrive - Black & Veatch/Documents/Jorge/Python/LLM Models

In [12]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("C:/Users/rod90731/Downloads/test", glob="**/*.docx")
books = loader.load()
len(books)

1

### Words Documents:

In [14]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("C:/Users/rod90731/OneDrive - Black & Veatch/Documents/Jorge/Python/LLM Models", glob="**/*.docx")
books = loader.load()
len(books)

1

### Text Documents:

In [48]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("C:/Users/rod90731/OneDrive - Black & Veatch/Documents/Jorge/Python/LLM Models", glob="Correction of Errors - APAC Performance (2.1)*.docx")
books = loader.load()
len(books)

1

In [49]:
books

[Document(page_content='Correction of Errors (COE)\n\nAPAC Teams, Internet, and ProjectWise Performance Issues Correction of Errors\n\n\u200d1.\xa0Description of problem and its impact\n\n\u200dDescription\n\nOver the past 18 months, and right after the RTO (Return to Office) Policy came up in January 2023, many of the professionals in the APAC region, and more specific in the Vikhroli, Pune and Bangkok offices started to notice and report performance issues while accessing the Internet, having Teams calls and accessing ProjectWise applications.\n\nBefore the COVID-19 Pandemic, most professionals in APAC and around the world did not have a good/reliable Internet service at home, as it was not the standard to work from home. During the Pandemic almost everybody invested in a good quality and reliable Internet Service at home, but the Office Bandwidth was not upgraded at all, and in some cases, the bandwidth was downgraded to save money on empty offices.\n\nThis huge discrepancy prior to

## Splitting the text

Feeding an entire book to the model is not feasible, as it would exceed its context window. To overcome this limitation, we must divide the text into smaller, more manageable chunks that fit comfortably within the model's context window.

In our case, we will convert all four books to a chunk size of 500 characters.

In [50]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
all_splits = text_splitter.split_documents(books)


In [51]:
all_splits

[Document(page_content='Correction of Errors (COE)\n\nAPAC Teams, Internet, and ProjectWise Performance Issues Correction of Errors\n\n\u200d1.\xa0Description of problem and its impact\n\n\u200dDescription\n\nOver the past 18 months, and right after the RTO (Return to Office) Policy came up in January 2023, many of the professionals in the APAC region, and more specific in the Vikhroli, Pune and Bangkok offices started to notice and report performance issues while accessing the Internet, having Teams calls and accessing ProjectWise applications.\n\nBefore the COVID-19 Pandemic, most professionals in APAC and around the world did not have a good/reliable Internet service at home, as it was not the standard to work from home. During the Pandemic almost everybody invested in a good quality and reliable Internet Service at home, but the Office Bandwidth was not upgraded at all, and in some cases, the bandwidth was downgraded to save money on empty offices.\n\nThis huge discrepancy prior to

## Ollama embeddings and Chroma vector store

We will use Langchain to convert the text into the embedding and store it in the Chroma database.

We are using the Ollama Llama 3 model as an embedding model.


In [52]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(model="llama3", show_progress=True),
    persist_directory="./chroma_db",
)

OllamaEmbeddings: 100%|██████████| 6/6 [10:37<00:00, 106.19s/it]


Let's test our vector database by performing some similarity searches.

In [54]:
question = "Who is Infosys?"
question = "10.10.7.11?"
question = "Why APAC performance issues?"
docs = vectorstore.similarity_search(question)
docs

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.88s/it]


[Document(page_content='Having Key resources in APAC will allow us to have a continuous support in the key areas to make Black & Veatch at the professionals successful.', metadata={'source': 'C:\\Users\\rod90731\\OneDrive - Black & Veatch\\Documents\\Jorge\\Python\\LLM Models\\Future of Infosys - Proposal -Final.docx'}),
 Document(page_content='FTE can only support legacy technologies with few exceptions, while the limited B&V employees are the one ending up supporting them.', metadata={'source': 'C:\\Users\\rod90731\\OneDrive - Black & Veatch\\Documents\\Jorge\\Python\\LLM Models\\Future of Infosys - Proposal -Final.docx'}),
 Document(page_content='Another side effect we have seen, is that people are working overtime to compensate from this attitude which is reflected in frustration and not exceptionally good customer service experiences by Black & Veatch Employees.', metadata={'source': 'C:\\Users\\rod90731\\OneDrive - Black & Veatch\\Documents\\Jorge\\Python\\LLM Models\\Future of I

## Building Langchain chains for Q&A retrieval system

To build a proper question-and-answer retrieval system, we will use Langchain chains and start adding the modules.

In our Q&A chain, we will

* Use vector store as the retriever and format the results.
* After that, we will provide the RAG prompt. You can easily pull that from the Langchain Hub.
* Then, we will provide the Ollama Llama 3 inference function.
* In the end, we will parse the results only to display the response.

Simply put, before passing it through the Llama 3 model, your question will be provided with context using the similarity search and RAG prompt.


In [55]:
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = Ollama(model="llama3")

retriever = vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_prompt = hub.pull("rlm/rag-prompt")
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

ConnectionError: HTTPSConnectionPool(host='api.hub.langchain.com', port=443): Max retries exceeded with url: /commits/rlm/rag-prompt/latest (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E87274EED0>: Failed to establish a new connection: [WinError 10013] An attempt was made to access a socket in a way forbidden by its access permissions'))

## Testing the Q&A retrieval chain

Ask relevant questions about books to understand more about the story.


In [35]:
question = "What is the story of Infosys?"
qa_chain.invoke(question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.27s/it]


"Infosys is an Indian IT company that provides primary IT management services globally, including help desk support, PC support, and infrastructure services like network and cloud. They have been providing these services to Black & Veatch for several years, but their capabilities no longer align with the company's current organizational structure and vision for the next 5 years. As a result, Black & Veatch has had to bring in external companies with subject matter experts to help overcome issues with Infosys, resulting in additional expenses."

In [22]:
question = "What is the story of Black & Veatch?"
qa_chain.invoke(question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:12<00:00, 12.50s/it]


'The story of Black & Veatch is not explicitly mentioned in the provided context. However, it seems that Black & Veatch is an organization that provides infrastructure services and has a global presence, with the goal of ensuring high-quality service delivery through strategic resource allocation and management. The context suggests that the company aims to improve its performance by leveraging key resources in Asia-Pacific (APAC) and optimizing its operations for better customer experiences.'

In [38]:
question = "where interface GigabitEthernet0/0/2.12 used?"
qa_chain.invoke(question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:03<00:00,  3.98s/it]


'The interface GigabitEthernet0/0/2.12 is used on the platform SGDCRTINET01.'

In [56]:
question = "why we had APAC Performance Issues?"
qa_chain.invoke(question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.42s/it]


'Based on the provided context, it appears that APAC Performance Issues are likely due to the limited availability of B&V employees who are burdened with supporting legacy technologies, leading to frustration and negative impacts on customer service. This situation may have contributed to overtime work, resulting in decreased morale and overall performance.'